![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev&file=dataproc_lr.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/dataproc_lr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fdataproc_lr.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/dataproc_lr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/dataproc_lr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Dataproc Spark Job
- Dataproc Cluster
- Job with BQ data
- Delete Dataproc Cluster

API Reference: https://googleapis.dev/python/dataproc/0.7.0/gapic/v1/api.html

## Setup

inputs:

In [29]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = 'dataproc'

DATAPROC_COMPUTE = "n1-standard-4"
DATAPROC_MAIN_INSTANCES = 1
DATAPROC_WORK_INSTANCES = 4

packages:

In [30]:
from google.cloud import dataproc_v1
from datetime import datetime

clients:

In [31]:
client_options = {"api_endpoint": f"{REGION}-dataproc.googleapis.com:443"}
clients = {}

In [41]:
clients['cluster'] = dataproc_v1.ClusterControllerClient(client_options = client_options)
clients['job'] = dataproc_v1.JobControllerClient(client_options = client_options)

parameters:

In [33]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [34]:
!rm -rf {DIR}
!mkdir -p {DIR}

E1004 08:58:28.045879212     133 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1633337908.045714837","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":321,"referenced_errors":[{"created":"@1633337908.045707029","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":957,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


## Create Cluster
https://cloud.google.com/dataproc/docs/guides/create-cluster

In [36]:
cluster_specs = {
	"project_id": PROJECT_ID,
    "cluster_name": DATANAME,
    "config": {
    	"master_config": {"num_instances": DATAPROC_MAIN_INSTANCES, "machine_type_uri": DATAPROC_COMPUTE},
    	"worker_config": {"num_instances": DATAPROC_WORK_INSTANCES, "machine_type_uri": DATAPROC_COMPUTE}
    }
}

In [37]:
cluster = clients['cluster'].create_cluster(
    request = {
        "project_id": PROJECT_ID,
        "region": REGION,
        "cluster": cluster_specs
	}
)

In [38]:
cluster.result().cluster_name

'fraud'

## Define Job
- https://cloud.google.com/dataproc/docs/tutorials/bigquery-sparkml#run_a_linear_regression

In [100]:
%%writefile {DIR}/train.py
from __future__ import print_function
from pyspark.context import SparkContext
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql.session import SparkSession
# The imports, above, allow us to access SparkML features specific to linear
# regression as well as the Vectors types.


# Define a function that collects the features of interest
# (mother_age, father_age, and gestation_weeks) into a vector.
# Package the vector in a tuple containing the label (`weight_pounds`) for that
# row.
def vector_from_inputs(r):
  return (r["weight_pounds"], Vectors.dense(float(r["mother_age"]),
                                            float(r["father_age"]),
                                            float(r["gestation_weeks"]),
                                            float(r["weight_gain_pounds"]),
                                            float(r["apgar_5min"])))

sc = SparkContext()
spark = SparkSession(sc)

spark.conf.set('temporaryGcsBucket',"statmike-mlops")

# Read the data from BigQuery as a Spark Dataframe.
#natality_data = spark.read.format("bigquery").option("table", "natality_regression.regression_input").load()
natality_data = spark.read.format("bigquery").option("table", "bigquery-public-data.samples.natality").load()
    # Create a view so that Spark SQL queries can be run against the data.
natality_data.createOrReplaceTempView("natality")

# As a precaution, run a query in Spark SQL to ensure no NULL values exist.
sql_query = """
SELECT weight_pounds, mother_age, father_age, gestation_weeks, weight_gain_pounds, apgar_5min
from natality
where weight_pounds is not null
and mother_age is not null
and father_age is not null
and gestation_weeks is not null
and weight_gain_pounds is not null
and apgar_5min is not null
"""
clean_data = spark.sql(sql_query)

# Create an input DataFrame for Spark ML using the above function.
training_data = clean_data.rdd.map(vector_from_inputs).toDF(["label",
                                                             "features"])
training_data.cache()

# Construct a new LinearRegression object and fit the training data.
lr = LinearRegression(maxIter=5, regParam=0.2, solver="normal")
model = lr.fit(training_data)
# Print the model summary.
print("Coefficients:" + str(model.coefficients))
print("Intercept:" + str(model.intercept))
print("R^2:" + str(model.summary.r2))
model.summary.residuals.show()

# write data to BigQuery
model.summary.residuals.write.format('bigquery').option("table", "statmike-mlops.fraud.dataproc").save()

Overwriting temp/dataproc/train.py


In [101]:
!gsutil cp {DIR}/train.py {URI}/{TIMESTAMP}/train.py

Copying file://temp/dataproc/train.py [Content-Type=text/x-python]...
/ [1 files][  2.5 KiB/  2.5 KiB]                                                
Operation completed over 1 objects/2.5 KiB.                                      


## Submit Job
- https://cloud.google.com/dataproc/docs/samples/dataproc-submit-pyspark-job

In [115]:
job_specs = {
	"placement": {"cluster_name": DATANAME},
    "pyspark_job": {
    	"main_python_file_uri": f"{URI}/{TIMESTAMP}/train.py",
        "jar_file_uris": ["gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar"]
    }
}

In [116]:
job = clients['job'].submit_job(project_id = PROJECT_ID, region = REGION, job = job_specs)

In [117]:
job.reference.job_id

'34d2b207-bc6b-4307-ba0b-5c8b6fce0319'

## Wait On Job

In [118]:
while True:
    ljob = clients['job'].get_job(project_id = PROJECT_ID, region = REGION, job_id = job.reference.job_id)
    if ljob.status.state.name == "ERROR":
        raise Exception(ljob.status.details)
    elif ljob.status.state.name == "DONE":
        print ("Finished")
        break

Finished


## Review Results
- Go to BiqQuery and review the output table: statmike-mlops.fraud.gm_cluster in my case

In [ ]:
ljob

## Delete Cluster
https://cloud.google.com/dataproc/docs/guides/manage-cluster#delete_a_cluster

In [120]:
delCluster = clients['cluster'].delete_cluster(
    request = {
        "project_id": PROJECT_ID,
        "region": REGION,
        "cluster_name": cluster.result().cluster_name
	}
)